In [1]:
# Install necessary packages
#!pip install numpy matplotlib ipywidgets scipy

import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import freqz, square, sawtooth
from IPython.display import Audio, display
import ipywidgets as widgets

# Generate white noise
fs = 44100  # Sampling frequency
duration = 2  # Duration in seconds
n_samples = duration * fs

white_noise = np.random.normal(0, 1, n_samples)

def low_pass_filter(signal, n_additions):
    """Simple moving average low-pass filter using only additions."""
    if n_additions < 1:
        return signal
    kernel = np.ones(n_additions)  # Create kernel of ones for moving average
    return np.convolve(signal, kernel, 'same') / n_additions

def plot_frequency_response(n_additions):
    """Plot the frequency response of the low-pass filter."""
    kernel = np.ones(n_additions)
    w, h = freqz(kernel, worN=8000)
    plt.figure(figsize=(12, 6))
    plt.plot(0.5*fs*w/np.pi, np.abs(h), 'b')
    plt.title(f'Frequency Response of Low-Pass Filter with {n_additions} Additions')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Gain')
    plt.grid()
    plt.show()

def generate_tone(freq, waveform, duty=0.5):
    """Generate a tone with the specified frequency and waveform."""
    t = np.linspace(0, duration, n_samples, endpoint=False)
    if waveform == 'Sine':
        return np.sin(2 * np.pi * freq * t)
    elif waveform == 'Triangle':
        return sawtooth(2 * np.pi * freq * t, 0.5)
    elif waveform == 'Square':
        return square(2 * np.pi * freq * t, duty)
    elif waveform == 'Sawtooth':
        return sawtooth(2 * np.pi * freq * t)
    else:
        return np.zeros(n_samples)

def plot_signals(unfiltered, filtered, freq):
    """Plot the unfiltered and filtered signals, showing only two periods."""
    period = int(fs / freq)
    t = np.linspace(0, 2 * period / fs, 2 * period, endpoint=False)
    
    plt.figure(figsize=(14, 6))
    plt.subplot(2, 1, 1)
    plt.plot(t, unfiltered[:2 * period])
    plt.title('Unfiltered Signal')
    plt.xlabel('Time [s]')
    plt.ylabel('Amplitude')
    
    plt.subplot(2, 1, 2)
    plt.plot(t, filtered[:2 * period])
    plt.title('Filtered Signal')
    plt.xlabel('Time [s]')
    plt.ylabel('Amplitude')
    
    plt.tight_layout()
    plt.show()

def update_audio(n_additions, freq, waveform, duty):
    """Update the filter, plot, and play the processed audio."""
    tone = generate_tone(freq, waveform, duty)
    filtered_signal = low_pass_filter(tone, n_additions)
    plot_frequency_response(n_additions)
    plot_signals(tone, filtered_signal, freq)
    display(Audio(filtered_signal, rate=fs))

# Create the sliders and radio buttons
n_additions_slider = widgets.IntSlider(min=1, max=50, step=1, value=1, description='Additions:')
freq_slider = widgets.FloatSlider(min=50, max=5000, step=1, value=440, description='Frequency (Hz):')
waveform_selector = widgets.RadioButtons(options=['Sine', 'Triangle', 'Square', 'Sawtooth'], description='Waveform:')
duty_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.5, description='Duty Cycle (Square):')

# Hide the duty cycle slider by default
duty_slider.layout.visibility = 'hidden'

def on_waveform_change(change):
    if change['new'] == 'Square':
        duty_slider.layout.visibility = 'visible'
    else:
        duty_slider.layout.visibility = 'hidden'

waveform_selector.observe(on_waveform_change, names='value')

# Create the interactive widget
widgets.interactive(update_audio, n_additions=n_additions_slider, freq=freq_slider, waveform=waveform_selector, duty=duty_slider)


interactive(children=(IntSlider(value=1, description='Additions:', max=50, min=1), FloatSlider(value=440.0, de…

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import freqz, square, sawtooth
from IPython.display import Audio, display
import ipywidgets as widgets

# Generate white noise
fs = 44100  # Sampling frequency
duration = 2  # Duration in seconds
n_samples = duration * fs

white_noise = np.random.normal(0, 1, n_samples)

def low_pass_filter(signal, n_additions):
    """Simple moving average low-pass filter using only additions."""
    if n_additions < 1:
        return signal
    kernel = np.ones(n_additions)  # Create kernel of ones for moving average
    return np.convolve(signal, kernel, 'same') / n_additions

def recursive_sum_filter(signal, delay):
    """Recursive sum low-pass filter."""
    if delay < 1:
        raise ValueError("Delay must be greater than or equal to 1")
    
    delay_line = np.zeros(delay)
    output_signal = np.zeros_like(signal)
    reg = 0
    
    for i in range(len(signal)):
        delayed_value = delay_line[i % delay]
        delay_line[i % delay] = signal[i]
        reg = signal[i] - delayed_value + reg
        output_signal[i] = reg
    
    return output_signal

def plot_frequency_response(n_additions, delay):
    """Plot the frequency response of the low-pass and recursive sum filters."""
    kernel = np.ones(n_additions) / n_additions
    w, h_sma = freqz(kernel, worN=8000)
    
    b = [1] + [0]*(delay-1) + [-1]
    a = [1, -1]
    w, h_recursive = freqz(b, a, worN=8000)
    h_recursive /= delay
    
    plt.figure(figsize=(12, 6))
    plt.plot(0.5 * fs * w / np.pi, np.abs(h_sma), 'b', label='SMA Filter')
    plt.plot(0.5 * fs * w / np.pi, np.abs(h_recursive), 'r', label='Recursive Sum Filter')
    plt.title('Frequency Response of Low-Pass Filters')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Gain')
    plt.legend()
    plt.grid()
    plt.show()

def generate_tone(freq, waveform, duty=0.5):
    """Generate a tone with the specified frequency and waveform."""
    t = np.linspace(0, duration, n_samples, endpoint=False)
    if waveform == 'Sine':
        return np.sin(2 * np.pi * freq * t)
    elif waveform == 'Triangle':
        return sawtooth(2 * np.pi * freq * t, 0.5)
    elif waveform == 'Square':
        return square(2 * np.pi * freq * t, duty)
    elif waveform == 'Sawtooth':
        return sawtooth(2 * np.pi * freq * t)
    else:
        return np.zeros(n_samples)

def plot_signals(unfiltered, filtered_sma, filtered_recursive, freq):
    """Plot the unfiltered and filtered signals, showing only two periods."""
    period = int(fs / freq)
    t = np.linspace(0, 2 * period / fs, 2 * period, endpoint=False)
    
    plt.figure(figsize=(14, 9))
    plt.subplot(3, 1, 1)
    plt.plot(t, unfiltered[:2 * period])
    plt.title('Unfiltered Signal')
    plt.xlabel('Time [s]')
    plt.ylabel('Amplitude')
    
    plt.subplot(3, 1, 2)
    plt.plot(t, filtered_sma[:2 * period])
    plt.title('SMA Filtered Signal')
    plt.xlabel('Time [s]')
    plt.ylabel('Amplitude')
    
    plt.subplot(3, 1, 3)
    plt.plot(t, filtered_recursive[:2 * period])
    plt.title('Recursive Sum Filtered Signal')
    plt.xlabel('Time [s]')
    plt.ylabel('Amplitude')
    
    plt.tight_layout()
    plt.show()

def update_audio(n_additions, delay, freq, waveform, duty):
    """Update the filter, plot, and play the processed audio."""
    tone = generate_tone(freq, waveform, duty)
    filtered_sma = low_pass_filter(tone, n_additions)
    filtered_recursive = recursive_sum_filter(tone, delay)
    plot_frequency_response(n_additions, delay)
    plot_signals(tone, filtered_sma, filtered_recursive, freq)
    display(Audio(filtered_recursive, rate=fs))

# Create the sliders and radio buttons
n_additions_slider = widgets.IntSlider(min=1, max=50, step=1, value=1, description='Additions:')
delay_slider = widgets.IntSlider(min=1, max=50, step=1, value=1, description='Delay:')
freq_slider = widgets.FloatSlider(min=50, max=5000, step=1, value=440, description='Frequency (Hz):')
waveform_selector = widgets.RadioButtons(options=['Sine', 'Triangle', 'Square', 'Sawtooth'], description='Waveform:')
duty_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.5, description='Duty Cycle (Square):')

# Hide the duty cycle slider by default
duty_slider.layout.visibility = 'hidden'

def on_waveform_change(change):
    if change['new'] == 'Square':
        duty_slider.layout.visibility = 'visible'
    else:
        duty_slider.layout.visibility = 'hidden'

waveform_selector.observe(on_waveform_change, names='value')

# Create the interactive widget
widgets.interactive(update_audio, n_additions=n_additions_slider, delay=delay_slider, freq=freq_slider, waveform=waveform_selector, duty=duty_slider)


interactive(children=(IntSlider(value=1, description='Additions:', max=50, min=1), IntSlider(value=1, descript…